In [2]:
%pip install geopandas rasterio rasterstats shapely fiona ipyleaflet geemap
%pip install folium earthengine-api
%pip install scikit-learn

     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 3.3 MB 24.7 MB/s 
     |████████████████████████████████| 476 kB 59.6 MB/s 
     |████████████████████████████████| 6.3 MB 19.8 MB/s 
     |████████████████████████████████| 129 kB 69.5 MB/s 
     |████████████████████████████████| 1.2 MB 75.2 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 1.6 MB 60.0 MB/s 
     |████████████████████████████████| 219 kB 66.1 MB/s 
     |████████████████████████████████| 93 kB 1.4 MB/s 
     |████████████████████████████████| 76 kB 5.7 MB/s 
     |████████████████████████████████| 98 kB 8.4 MB/s 
     |████████████████████████████████| 1.3 MB 25.4 MB/s 
     |████████████████████████████████| 130 kB 65.7 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████

In [3]:

from os import path as op
import pickle
import fiona
import geemap
import ipyleaflet
import geopandas as gpd
import shapely as shp
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterstats.io import bounds_window
import rasterstats
import folium


In [4]:
!pip install geehydro
import geehydro
import folium

  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10141 sha256=5eb52e5b030bf5ad1249dde5b1f85236c6de1759fcc7a9171aecb9f0d175e9e2
  Stored in directory: /root/.cache/pip/wheels/49/6c/92/019aeb37663078e6666afa9dd336e2d8df72d5683d7d357697
Successfully built geehydro


In [5]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [6]:
!earthengine authenticate
import ee 
ee.Initialize()

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=XP7arGyw6rP91S0pL3SGxxFSNFZbmH2KZiayBrMRk-g&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgO-IPvOsX-6nV5QvqcpUMRd3aMnkBbeZLLFkS_QDoOeWfrfeVGnjo

Successfully saved authorization token.


In [7]:
c = fiona.open('/content/drive/My Drive/Colab Notebooks/data/river_shape/River_shape.shp')

polygons = []

for row in c:
  polygons.append(row['geometry']['coordinates'])

In [261]:
#Tapi river
aoi = ee.Geometry.Polygon(
         [[[73.54407919861856, 21.506806808887394],
          [73.54407919861856, 21.165269300114176],
          [74.040523900767, 21.165269300114176],
          [74.040523900767, 21.506806808887394]]])

In [262]:
def ndwi_f(img):
  ndwi=img.normalizedDifference(['B3','B8']).rename('NDWI')
  return img.addBands(ndwi).updateMask(ndwi.gt(0))

In [263]:
def ndci_f(img):
  #water mask
  ndci = img.normalizedDifference(['B5', 'B4']).rename('NDCI');
  return img.addBands(ndci)


In [292]:
band_sel = [ 'B3', 'B4', 'B5', 'B8']
def map_(start_date,end_date):
  IMG=ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(aoi).filterDate(start_date,end_date).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)).median().clip(aoi)
  IMG_water = ndwi_f(IMG)
  IMG_NDCI = ndci_f(IMG_water)
  Map=folium.Map(location=[   21.33,73.79 ],zoom_start=10)

  viz = {'min':0.02,'max':0.05,'palette':['cyan','orange','red']}

  Map.addLayer(IMG.clip(aoi),{'bands':['B4','B3','B2'],'min':0,'max':3500},'IMG')
  Map.addLayer(IMG_water.clip(aoi).select('NDWI'),{'palette':['cyan']},"IMG_water")
  Map.addLayer(IMG_NDCI.clip(aoi).select('NDCI'),viz,"IMG_NDCI")
  Map.setControlVisibility(layerControl=True, fullscreenControl=False, latLngPopup=True)
  return Map

In [298]:

march=map_('2020-03-01','2020-03-30')
april=map_('2020-04-01','2020-04-30')
may=map_('2020-05-01','2020-05-30')
dec=map_('2020-10-01','2020-12-30')



In [294]:

march

In [295]:
april

In [296]:
may


In [297]:
dec